# Flow-shop
Biel Nebot Cabrera
___
Només cal modificar els paràmetres que s'especifiquen als inputs de cada funció.

In [1]:
import pandas as pd
import numpy as np
import itertools

class Durada:
    def __init__(self,dura,peca,maquina):
        self.dura = dura
        self.peca = peca
        self.maquina = maquina

    def __str__(self):
        return "La peça {} dura {} a la màquina {}".format(self.peca,self.dura,self.maquina)

**Funcions complementàries**

In [2]:
def creaLlistaAmbMatriu(matriu):
    matDades = np.array(matriu) 
    files, columnes = matDades.shape
    
    llistaDades = []
    for j in range(columnes):
        for i in range(files):
            llistaDades.append(Durada(matDades[i][j],j,i))
#     [print(i) for i in llistaDades];
    return llistaDades

def diguesIndexDelMinim(llistaDades):
    # INPUT
    # Una llista on cada elements és un objecte de durada i,j
    # RETORNA:
    # - la posició on hi ha el minim i
    # - la màquina a que correspòn
    llistaDurades = [i.dura for i in llistaDades]
    valorMinim = min(llistaDurades)
    indexDelMinim = llistaDurades.index(valorMinim)
    return (indexDelMinim, llistaDades[indexDelMinim].maquina)

def eliminaColumnaAmbIndex(llistaDades,index):
    idPeca = llistaDades[index].peca
    return [i for i in llistaDades if i.peca != idPeca]

# COTA
def tallaLlistaPerIndex(ll,indx):
    # fet servir pel càlcul de la cota
    return ll[:indx], ll[indx], ll[indx+1:]

def sumaBlocsAbans_i_Despres(llAbans,llDespres):
    # fet servir pel càlcul de la cota
    matAbans = np.array(llAbans)
    matDespres = np.array(llDespres)
    return matAbans.sum(axis=0), matDespres.sum(axis=0)

def llista_p_ij_aConsiderar(sumaAnt,sumaSeg):
    # fet servir pel càlcul de la cota
    if isinstance(sumaAnt, np.float64):
        return min(sumaSeg)
    elif isinstance(sumaSeg, np.float64):
        return min(sumaAnt)
    else:
        llistaPossibles = []
        for peca in range(len(sumaAnt)):
            for parella in range(len(sumaAnt)):
                if peca != parella:
                    llistaPossibles.append(sumaAnt[peca]+sumaSeg[parella])
        return min(llistaPossibles)
# NEH 
def nousOrdres(ordreAntic,nouElement):
    llistaCombinacions = []
    for i in range(1+len(ordreAntic)):
        llistaCombinacions.append(afegeix_a_la_Posició_i(nouElement,ordreAntic,i))
    return llistaCombinacions

def afegeix_a_la_Posició_i(x,ll,i):
    return ll[:i] + [x] + ll[i:]

## Algoritme Johnson
Calen **dues** màquines.

**Input:**
- `nomsPeces` és una llista dels noms de les peces ordenada tal com apareixen a la taula.
- `taulaDades` és una llista de llistes que representa la taula de dades.
- `nomsMaquines` és una llista dels noms de les màquines ordenada tal com apareixen a la taula.

**Exemple:**

|                 |  A   |  B   |  C   |  D   |  E   |
| :-------------: | :--: | :--: | :--: | :--: | :--: |
|  **m1**  |  1   |  2   |  3   |  2   |  3   |
|  **m2**  |  3   |  5   |  1   |  7   |  4   |


- `nomsPeces = ["A","B","C","D","E"]`
- `taulaDades = [[1, 2, 3, 2, 3], [3, 5, 1, 7, 4]]`
- `nomsMaquines = ["m1","m2"]`

**Output:**
Peces ordenades

In [3]:
def Johnson(taulaDades,nomsPeces,nomsMaquines):
    # FEM UN PRINT
    print("Les dades d'entrada són:")
    df = pd.DataFrame(taulaDades); df.columns = nomsPeces; print(df)
    # FEM MATRIU DE LES DADES
    matDades = np.array(taulaDades) 
    files, columnes = matDades.shape
    print("\nHi ha {} màquines i {} peces".format(files,columnes))
    if files != 2:
        print("Només hi poden haver dues màquines per fer el Johnson!")
    else:
        # FEM LLISTA d'objectes DE LES DADES
        llistaDades = creaLlistaAmbMatriu(matDades)
#         [print(i) for i in llistaDades];
        llistaPrimeres = []
        llistaUltimes = []

        for iteracio in range(columnes):
#             print("\n##### Iteració ",iteracio," #####")
            indexMinim, idMaquina = diguesIndexDelMinim(llistaDades)
#             print("Mínim a la màquina {} i val {}".format(nomsMaquines[idMaquina],llistaDades[indexMinim].dura))
            if idMaquina == 0:
                llistaPrimeres.append(llistaDades[indexMinim].peca)
            elif idMaquina == 1:
                llistaUltimes.insert(0,llistaDades[indexMinim].peca)
            llistaDades = eliminaColumnaAmbIndex(llistaDades,indexMinim)

        llistaOrdre = llistaPrimeres + llistaUltimes
        print("\nResultat: ",[nomsPeces[i] for i in llistaOrdre])
        return llistaOrdre

    
# Exemple proposat
taulaDadesProva = [[1, 2, 3, 2, 3],
                   [3, 5, 1, 7, 4]]
nomsPecesProva = ["A","B","C","D","E"]
nomsMaquinesProva = ["m1","m2"]


# executa    
Johnson(taulaDadesProva,nomsPecesProva,nomsMaquinesProva);

Les dades d'entrada són:
   A  B  C  D  E
0  1  2  3  2  3
1  3  5  1  7  4

Hi ha 2 màquines i 5 peces

Resultat:  ['A', 'B', 'D', 'E', 'C']


## Horari acaba
**Input:**
- `nomsPeces` és una llista dels noms de les peces ordenada tal com apareixen a la taula.
- `taulaDades` és una llista de llistes que representa la taula de dades.
- `llistaOrdre` és una llista on hi ha les peces ordenades segons l'ordre en què s'han de fabricar. S'ordenen fent servir l'index de la columna on apareixen a la taula.

**Exemple:**

|                 |  A   |  B   |  C   |  D   |  E   |
| :-------------: | :--: | :--: | :--: | :--: | :--: |
|  **m1**  |  1   |  2   |  3   |  2   |  3   |
|  **m2**  |  3   |  5   |  1   |  7   |  4   |

Ordenades així: A > B > D > E > C

- `nomsPeces = ["A","B","C","D","E"]`
- `taulaDades = [[1, 2, 3, 2, 3], [3, 5, 1, 7, 4]]`
- `llistaOrdre = [0, 1, 3, 4, 2]`

**Output:**
Taula amb l'instant en què cada peça surt de cada màquina.

In [4]:
def taulaPecesAcaben(llistaOrdre,taulaDades,nomsPeces):
    matDades = np.array(taulaDades) 
    files, columnes = matDades.shape
    
    llistaResultats = []
    
    for maquina in range(files):
        llistaMaquina = []
        ultim = 0
        comptadorColumna = 0
        for peca in llistaOrdre:
            if maquina == 0:
                perAfegir = matDades[maquina][peca]
                ultim += perAfegir
            else:
                ultim = matDades[maquina][peca] + max(ultim,llistaResultats[-1][comptadorColumna])           
                comptadorColumna += 1
            llistaMaquina.append(ultim)
        llistaResultats.append(llistaMaquina)
        
    df = pd.DataFrame(llistaResultats); df.columns = [nomsPeces[i] for i in llistaOrdre]; print(df)
    return llistaResultats


# Exemple proposat
llOrdreProva = [0, 1, 3, 4, 2]
taulaDadesProva = [[1, 2, 3, 2, 3],
                   [3, 5, 1, 7, 4]]
nomsPecesProva = ["A","B","C","D","E"]


# executa
a = taulaPecesAcaben(llOrdreProva,taulaDadesProva,nomsPecesProva);

   A  B   D   E   C
0  1  3   5   8  11
1  4  9  16  20  21


## Càlcul de la cota
**Input:**
- `nomsPeces` és una llista dels noms de les peces ordenada tal com apareixen a la taula.
- `taulaDades` és una llista de llistes que representa la taula de dades.
- `nomsMaquines` és una llista dels noms de les màquines ordenades tal com apareixen a la taula.

**Exemple:**

|                 |  A   |  B   |  C   |  D   |  E   |
| :-------------: | :--: | :--: | :--: | :--: | :--: |
|  **m1**  |  1   |  2   |  3   |  2   |  3   |
|  **m2**  |  3   |  5   |  1   |  7   |  4   |
|  **m3**  |  2   |  3   |  1   |  6   |  1   |

- `nomsPeces = ["A","B","C","D","E"]`
- `taulaDades = [[1, 2, 3, 2, 3], [3, 5, 1, 7, 4], [2, 3, 1, 6, 1]]`
- `nomsMaquines = ["m1", "m2", "m3"]`

**Output:**
Valor de K

In [5]:
def calculaCotes(taulaDades,nomsPeces,nomsMaquines):
    print("########## COMENÇA CÀLCUL DE LA COTA ##########")
    # FEM UN PRINT
    df = pd.DataFrame(taulaDades); df.columns = nomsPeces; print(df)
    # FEM MATRIU DE LES DADES
    matDades = np.array(taulaDades) 
    files, columnes = matDades.shape
    
    llista_K = []    
    
    for maquina in range(files):
        print("\n##### Màquina {} #####".format(nomsMaquines[maquina]))
        llistaPossibles = []
        anterior, actual, seguent = tallaLlistaPerIndex(taulaDades,maquina)
        p_ij = sum(actual)
        print("p_ij = ",p_ij)
        sumaAnterior, sumaSeguent = sumaBlocsAbans_i_Despres(anterior,seguent)
        print("Suma anterior = ",sumaAnterior,type(sumaAnterior))
        print("Suma seguent = ",sumaSeguent,type(sumaSeguent))
        minLlista = llista_p_ij_aConsiderar(sumaAnterior,sumaSeguent)
        print("El mínim és ",minLlista)
    
        llista_K.append(p_ij+minLlista)
    print("\nCota global K = max",llista_K,"=",max(llista_K))
    return max(llista_K)


# Exemple proposat
nomsMaquinesEnunciatProva = ["m1", "m2", "m3"]
dadesEnunciatProva = [[1, 2, 3, 2, 3],
                      [3, 5, 1, 7, 4],
                      [2, 3, 1, 6, 1]]
nomsPecesEnunciatProva = ["A","B","C","D","E"]


# Exemple amb més dades
# nomsMaquinesEnunciatProva = ["m1", "m2", "m3","m4","m5","m6"]
# dadesEnunciatProva = [[1, 2, 3, 2, 3, 5, 1, 7],
#                       [3, 5, 1, 7, 4, 9, 2, 8],
#                       [2, 3, 1, 6, 1, 8, 3, 6],
#                       [4, 8, 5, 8, 9, 8, 4, 9],
#                       [7, 2, 6, 3, 9, 6, 5, 1],
#                       [2, 1, 6, 9, 3, 5, 6, 7]]
# nomsPecesEnunciatProva = ["A","B","C","D","E","F","G","H"]


# executa
calculaCotes(dadesEnunciatProva,nomsPecesEnunciatProva,nomsMaquinesEnunciatProva)

########## COMENÇA CÀLCUL DE LA COTA ##########
   A  B  C  D  E
0  1  2  3  2  3
1  3  5  1  7  4
2  2  3  1  6  1

##### Màquina m1 #####
p_ij =  11
Suma anterior =  0.0 <class 'numpy.float64'>
Suma seguent =  [ 5  8  2 13  5] <class 'numpy.ndarray'>
El mínim és  2

##### Màquina m2 #####
p_ij =  20
Suma anterior =  [1 2 3 2 3] <class 'numpy.ndarray'>
Suma seguent =  [2 3 1 6 1] <class 'numpy.ndarray'>
El mínim és  2

##### Màquina m3 #####
p_ij =  13
Suma anterior =  [4 7 4 9 7] <class 'numpy.ndarray'>
Suma seguent =  0.0 <class 'numpy.float64'>
El mínim és  4

Cota global K = max [13, 22, 17] = 22


22

## Màquines virtuals
**Input:**
- `nomsPeces` és una llista dels noms de les peces ordenada tal com apareixen a la taula.
- `taulaDades` és una llista de llistes que representa la taula de dades.

**Exemple:**

|                 |  A   |  B   |  C   |  D   |  E   |
| :-------------: | :--: | :--: | :--: | :--: | :--: |
|  **Màquina 1**  |  1   |  2   |  3   |  2   |  3   |
|  **Màquina 2**  |  3   |  5   |  1   |  7   |  4   |
|  **Màquina 3**  |  2   |  3   |  1   |  6   |  1   |

- `nomsPeces = ["A","B","C","D","E"]`
- `taulaDades = [[1, 2, 3, 2, 3], [3, 5, 1, 7, 4], [2, 3, 1, 6, 1]]`

**Output:**
Taula amb els paràmetres $s_1$, $s_2$ i $s_3$ per després aplicar Palmer o Trapezis:

|                 |  A   |  B   |  C   |  D   |  E   |
| :-------------: | :--: | :--: | :--: | :--: | :--: |
|  **s1**  |  ...   |  ...   |  ...   |  ...   |  ...   |
|  **s2**  |  ...   |  ...   |  ...   |  ...   |  ...   |
|  **s3**  |  ...   |  ...   |  ...   |  ...   |  ...   |

In [6]:
def maquinesVirtuals(taulaDades,nomsPeces):
    print("########## COMENÇA CÀLCUL D'S1, S2 i S3 ##########")
    # FEM UN PRINT
    df = pd.DataFrame(taulaDades); df.columns = nomsPeces; print(df)
    # FEM MATRIU DE LES DADES
    matDades = np.array(taulaDades) 
    files, columnes = matDades.shape
    
    llista_Resultats = []    
    
    for peca in range(columnes):
        llesca = matDades[:,peca]
        S1 = sum([llesca[i-1]*(files-i) for i in range(1,files+1)])
        S2 = sum([llesca[i-1]*(i-1) for i in range(1,files+1)])
        S3 = S1 - S2
        llista_Resultats.append([S1, S2, S3])
        
    llista_Resultats = np.array(llista_Resultats).transpose()
    print("\nResultat S1, S2 i S3:")
    df = pd.DataFrame(llista_Resultats); df.columns = nomsPeces; print(df)    
    return llista_Resultats


# Exemple proposat
dadesEnunciatProva = [[1, 2, 3, 2, 3],
                      [3, 5, 1, 7, 4],
                      [2, 3, 1, 6, 1]]
nomsPecesEnunciatProva = ["A","B","C","D","E"]


# Exemple amb més dades
# dadesEnunciatProva = [[1, 2, 3, 2, 3, 5, 1, 7],
#                       [3, 5, 1, 7, 4, 9, 2, 8],
#                       [2, 3, 1, 6, 1, 8, 3, 6],
#                       [4, 8, 5, 8, 9, 8, 4, 9],
#                       [7, 2, 6, 3, 9, 6, 5, 1],
#                       [2, 1, 6, 9, 3, 5, 6, 7]]
# nomsPecesEnunciatProva = ["A","B","C","D","E","F","G","H"]


# executa
maquinesVirtuals(dadesEnunciatProva,nomsPecesEnunciatProva);

########## COMENÇA CÀLCUL D'S1, S2 i S3 ##########
   A  B  C  D  E
0  1  2  3  2  3
1  3  5  1  7  4
2  2  3  1  6  1

Resultat S1, S2 i S3:
   A   B  C   D   E
0  5   9  7  11  10
1  7  11  3  19   6
2 -2  -2  4  -8   4


## NEH
**Input:**
- `nomsPeces` és una llista dels noms de les peces ordenada tal com apareixen a la taula.
- `taulaDades` és una llista de llistes que representa la taula de dades.
- `llOrdre` és una llista que ordena les peces de més a menys temps total de fabricació fent servir l'index de la colmna on estan.

**Exemple:**

|                 |  A   |  B   |  C   |  D   |  E   |
| :-------------: | :--: | :--: | :--: | :--: | :--: |
|  **Màquina 1**  |  1   |  2   |  3   |  2   |  3   |
|  **Màquina 2**  |  3   |  5   |  1   |  7   |  4   |
|  **Màquina 3**  |  2   |  3   |  1   |  6   |  1   |
| |    |   |  |  |    |
| **Temps total** |  6   |  10  |  5   |  15  |  8   |

- `nomsPeces = ["A","B","C","D","E"]`
- `taulaDades = [[1, 2, 3, 2, 3], [3, 5, 1, 7, 4], [2, 3, 1, 6, 1]]`

I ordenades de més a menys temps queda `[D, B, E, A, C]`. Per tant:

- `llOrdre = [3, 1, 4, 0, 2]`

In [7]:
def NEH(llOrdre,taulaDades,nomsPeces):
    nivellSuperior = llOrdre[:1]
    for iteracio in range(1,len(llOrdre)):
        print("\n###### Iteració ",iteracio," ######")
        print("Nivell superior és: ",[nomsPeces[p] for p in nivellSuperior]," i el nou és ",nomsPeces[llOrdre[iteracio]])
        
        combinacionsPerProvar = nousOrdres(nivellSuperior,llOrdre[iteracio])
        llAmbNoms = []
        parellaDuraMinim = (0,1e16)
        for k in combinacionsPerProvar:
            llNoms = [nomsPeces[j] for j in k]
            print("\n Nova combinació: ",llNoms)
            llAmbNoms.append(llNoms)
            res = taulaPecesAcaben(k,taulaDades,nomsPeces)
            horaAcaba = res[-1][-1]
            
            if parellaDuraMinim[1] > horaAcaba:
                parellaDuraMinim = (k,horaAcaba)
#         print("\n",combinacionsPerProvar,"\n",llAmbNoms)
        
        nivellSuperior = parellaDuraMinim[0]
        
    print("\nLa millor assolible amb NEH és ",parellaDuraMinim)
    return parellaDuraMinim


# Exemple proposat
nomsPecesProva = ["A","B","C","D","E"]
llOrdreProva = [3, 1, 4, 0, 2]
taulaDadesProva = [[1, 2, 3, 2, 3],
                   [3, 5, 1, 7, 4],
                   [2, 3, 1, 6, 1]]


# Exemple amb més dades
# taulaDadesProva = [[1, 2, 3, 2, 3, 5, 1, 7],
#                       [3, 5, 1, 7, 4, 9, 2, 8],
#                       [2, 3, 1, 6, 1, 8, 3, 6],
#                       [4, 8, 5, 8, 9, 8, 4, 9],
#                       [7, 2, 6, 3, 9, 6, 5, 1],
#                       [2, 1, 6, 9, 3, 5, 6, 7]]
# nomsPecesProva = ["A","B","C","D","E","F","G","H"]
# llOrdreProva = [5, 7, 3, 4, 2, 1, 6, 0]


# executa
NEH(llOrdreProva,taulaDadesProva,nomsPecesProva);


###### Iteració  1  ######
Nivell superior és:  ['D']  i el nou és  B

 Nova combinació:  ['B', 'D']
    B   D
0   2   4
1   7  14
2  10  20

 Nova combinació:  ['D', 'B']
    D   B
0   2   4
1   9  14
2  15  18

###### Iteració  2  ######
Nivell superior és:  ['D', 'B']  i el nou és  E

 Nova combinació:  ['E', 'D', 'B']
   E   D   B
0  3   5   7
1  7  14  19
2  8  20  23

 Nova combinació:  ['D', 'E', 'B']
    D   E   B
0   2   5   7
1   9  13  18
2  15  16  21

 Nova combinació:  ['D', 'B', 'E']
    D   B   E
0   2   4   7
1   9  14  18
2  15  18  19

###### Iteració  3  ######
Nivell superior és:  ['D', 'B', 'E']  i el nou és  A

 Nova combinació:  ['A', 'D', 'B', 'E']
   A   D   B   E
0  1   3   5   8
1  4  11  16  20
2  6  17  20  21

 Nova combinació:  ['D', 'A', 'B', 'E']
    D   A   B   E
0   2   3   5   8
1   9  12  17  21
2  15  17  20  22

 Nova combinació:  ['D', 'B', 'A', 'E']
    D   B   A   E
0   2   4   5   8
1   9  14  17  21
2  15  18  20  22

 Nova combinació:  ['D